In [0]:
# Databricks notebook source
from pyspark.sql.functions import col, desc
import os

# COMMAND ----------

# Configuração dos volumes a serem limpos
volumes = [
    "/Volumes/bitz/raw/saldo",
    "/Volumes/bitz/raw/contas_receber_fcp",
    "/Volumes/bitz/raw/contas_pagar_fcp"
]

# Número de arquivos mais recentes a manter
KEEP_LATEST = 4

# COMMAND ----------

def cleanup_volume(volume_path, keep_count=4):
    """
    Mantém apenas os N arquivos mais recentes no volume e deleta o restante.
    
    Args:
        volume_path: Caminho do volume no formato /Volumes/catalog/schema/volume
        keep_count: Número de arquivos mais recentes a manter (padrão: 5)
    
    Returns:
        dict: Estatísticas da limpeza (total, mantidos, deletados)
    """
    try:
        # Lista todos os arquivos no volume
        files = dbutils.fs.ls(volume_path)
        
        # Filtra apenas arquivos (não diretórios) e ordena por data de modificação
        file_list = [
            (f.path, f.modificationTime, f.size) 
            for f in files 
            if not f.isDir()
        ]
        
        # Ordena por data de modificação (mais recente primeiro)
        file_list_sorted = sorted(file_list, key=lambda x: x[1], reverse=True)
        
        total_files = len(file_list_sorted)
        
        # Separa arquivos a manter e a deletar
        files_to_keep = file_list_sorted[:keep_count]
        files_to_delete = file_list_sorted[keep_count:]
        
        # Deleta os arquivos antigos
        deleted_count = 0
        deleted_size = 0
        
        for file_path, mod_time, size in files_to_delete:
            try:
                dbutils.fs.rm(file_path)
                deleted_count += 1
                deleted_size += size
                print(f"✓ Deletado: {file_path}")
            except Exception as e:
                print(f"✗ Erro ao deletar {file_path}: {str(e)}")
        
        # Estatísticas
        stats = {
            "volume": volume_path,
            "total_files": total_files,
            "kept": len(files_to_keep),
            "deleted": deleted_count,
            "deleted_size_mb": round(deleted_size / (1024 * 1024), 2)
        }
        
        return stats
        
    except Exception as e:
        print(f"Erro ao processar volume {volume_path}: {str(e)}")
        return None

# COMMAND ----------

# Executa a limpeza para cada volume
print("=" * 80)
print(f"INICIANDO LIMPEZA DOS VOLUMES - Mantendo os {KEEP_LATEST} arquivos mais recentes")
print("=" * 80)
print()

all_stats = []

for volume in volumes:
    print(f"\n📁 Processando: {volume}")
    print("-" * 80)
    
    stats = cleanup_volume(volume, KEEP_LATEST)
    
    if stats:
        all_stats.append(stats)
        print(f"\n📊 Resumo:")
        print(f"   Total de arquivos: {stats['total_files']}")
        print(f"   Mantidos: {stats['kept']}")
        print(f"   Deletados: {stats['deleted']}")
        print(f"   Espaço liberado: {stats['deleted_size_mb']} MB")
    
    print("-" * 80)

# COMMAND ----------

# Resumo geral
print("\n" + "=" * 80)
print("RESUMO GERAL DA LIMPEZA")
print("=" * 80)

total_deleted = sum(s['deleted'] for s in all_stats)
total_space_freed = sum(s['deleted_size_mb'] for s in all_stats)

print(f"\n✅ Volumes processados: {len(all_stats)}")
print(f"✅ Total de arquivos deletados: {total_deleted}")
print(f"✅ Espaço total liberado: {total_space_freed:.2f} MB")
print("\n" + "=" * 80)